In [0]:
#english to japanese SEQ2SEQ model

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [1]:
!pwd
%cd ./gdrive/My Drive/Projects/Colab Notebooks/Keras-SEQ2SEQ-NMT-english-Japanese-master

/content
/content/gdrive/My Drive/Projects/Colab Notebooks/Keras-SEQ2SEQ-NMT-english-Japanese-master


In [2]:
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
from keras.callbacks import ModelCheckpoint
import nltk
import numpy as np
import os
import zipfile
import sys
import urllib.request
from gensim.models import KeyedVectors
from keras.utils import plot_model


Using TensorFlow backend.


In [0]:
from keras.activations import softmax
from keras.layers.core import Dense, Activation, RepeatVector, Permute
from keras.layers import Input, Embedding, Multiply, Concatenate, Lambda
from keras.layers.wrappers import TimeDistributed

#hyperparameters

In [0]:
BATCH_SIZE = 64
NUM_EPOCHS = 100
HIDDEN_UNITS = 256
NUM_SAMPLES = 10000
MAX_VOCAB_SIZE = 2000
EMBEDDING_SIZE = 100
DATA = 'jpn.txt'#loading data from default work directory

In [0]:
tar_count = Counter()

GLOVE_MODEL = "glove.6B.100d.txt"
WEIGHT_FILE_PATH = 'eng-to-jpn-glove-weights.h5'
MODEL_FILE_PATH = 'eng-to-jpn-glove-model.h5'


In [0]:
def load_glove():
    
    _word2em = {}
    file = open(GLOVE_MODEL, mode='r', encoding='utf8')
    for line in file:
        words = line.strip().split()
        word = words[0]
        embeds = np.array(words[1:], dtype=np.float32)
        _word2em[word] = embeds
    file.close()
    return _word2em

In [0]:
word2em = load_glove()

In [0]:
lines = open(DATA, 'r', encoding='utf8').read().split('\n')
for line in lines[: len(lines)-1]:
    input_text, target_text = line.split('\t')
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    target_text = '\t' + target_text + '\n'
    for char in target_text:
        tar_count[char] += 1


In [0]:
target_word2idx = dict()

for idx, word in enumerate(tar_count.most_common(MAX_VOCAB_SIZE)):
    #print(word)
    target_word2idx[word[0]] = idx

target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_decoder_tokens = len(target_idx2word)

In [0]:
unknown_emb = np.random.randn(EMBEDDING_SIZE)

In [0]:
encoder_max_seq_length = 0
decoder_max_seq_length = 0

input_texts_word2em = []


In [0]:
#encoder word2index input

In [0]:
lines = open(DATA, 'r', encoding='utf8').read().split('\n')
for line in lines[: min(NUM_SAMPLES, len(lines)-1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    encoder_input_wids = []
    for w in input_words:
        em = unknown_emb
        if w in word2em:
            em = word2em[w]
        encoder_input_wids.append(em)
    input_texts_word2em.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_text), decoder_max_seq_length)

In [0]:
encoder_input_data = pad_sequences(input_texts_word2em, encoder_max_seq_length)

In [0]:
#decoder word2index input

In [0]:
decoder_target_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
decoder_input_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
lines = open(DATA, 'rt', encoding='utf8').read().split('\n')
for lineIdx, line in enumerate(lines[: min(NUM_SAMPLES, len(lines)-1)]):
    _, target = line.split('\t')
    target = '\t' + target + '\n'
    for idx, char in enumerate(target):
        if char in target_word2idx:
            w2idx = target_word2idx[char]
            decoder_input_data[lineIdx, idx, w2idx] = 1
            if idx > 0:
                decoder_target_data[lineIdx, idx-1, w2idx] = 1


In [0]:
context = dict()
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length


In [0]:
#defining Encoder- Decoder Model

In [19]:
encoder_inputs = Input(shape=(None, EMBEDDING_SIZE), name='encoder_inputs')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss='categorical_crossentropy', optimizer='adam')



checkpoint = ModelCheckpoint(filepath=WEIGHT_FILE_PATH, save_best_only=True)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
          verbose=1, validation_split=0.3, callbacks=[checkpoint])

model.save_weights(WEIGHT_FILE_PATH)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 7000 samples, validate on 3000 samples
Epoch 1/100
7000/7000 [==============================] - 21s 3ms/step - loss: 1.5155 - val_loss: 1.7014


/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_lstm/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'encoder_lstm/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/100
7000/7000 [==============================] - 16s 2ms/step - loss: 1.3416 - val_loss: 1.6419
Epoch 3/100
7000/7000 [==============================] - 16s 2ms/step - loss: 1.2775 - val_loss: 1.5742
Epoch 4/100
7000/7000 [==============================] - 16s 2ms/step - loss: 1.1970 - val_loss: 1.4764
Epoch 5/100
7000/7000 [==============================] - 16s 2ms/step - loss: 1.1121 - val_loss: 1.3913
Epoch 6/100
7000/7000 [==============================] - 17s 2ms/step - loss: 1.0392 - val_loss: 1.3296
Epoch 7/100
7000/7000 [==============================] - 16s 2ms/step - loss: 0.9816 - val_loss: 1.2742
Epoch 8/100
7000/7000 [==============================] - 16s 2ms/step - loss: 0.9311 - val_loss: 1.2340
Epoch 9/100
7000/7000 [==============================] - 16s 2ms/step - loss: 0.8875 - val_loss: 1.1988
Epoch 10/100
7000/7000 [==============================] - 16s 2ms/step - loss: 0.8472 - val_loss: 1.1684
Epoch 11/100
7000/7000 [==============================] - 17s 2

In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, None, 100)    0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (None, None, 2000)   0                                            
__________________________________________________________________________________________________
encoder_lstm (LSTM)             [(None, 256), (None, 365568      encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_lstm (LSTM)             [(None, None, 256),  2311168     decoder_inputs[0][0]             
                                                                 encoder_lstm[0][1]               
          

In [0]:
#Model to Test

In [0]:
# Encoder inference model
encoder_model_inf = Model(encoder_inputs, encoder_states)

# Decoder inference model

decoder_state_input_h = Input(shape=(HIDDEN_UNITS,))
decoder_state_input_c = Input(shape=(HIDDEN_UNITS,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_lstm(decoder_inputs, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_inputs] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [0]:
max_encoder_seq_length = context['encoder_max_seq_length']
max_decoder_seq_length = context['decoder_max_seq_length']
num_decoder_tokens = context['num_decoder_tokens']

In [0]:
def predict_sent(input_text):
        input_seq = []
        input_wids = []
        for word in nltk.word_tokenize(input_text.lower()):
            emb = unknown_emb
            if word in word2em:
                emb = word2em[word]
            input_wids.append(emb)
        input_seq.append(input_wids)
        input_seq = pad_sequences(input_seq, max_encoder_seq_length)
        states_value = encoder_model_inf.predict(input_seq)
        target_seq = np.zeros((1, 1,num_decoder_tokens))
        target_seq[0, 0, target_word2idx['\t']] = 1
        target_text = ''
        terminated = False
        while not terminated:
            output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = target_idx2word[sample_token_idx]
            target_text += sample_word

            if sample_word == '\n' or len(target_text) >= max_decoder_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip()

In [32]:
print(predict_sent('He is sick.'))

彼は病気です。


In [29]:
# download model image file
from google.colab import files
files.download(MODEL_FILE_PATH)

FileNotFoundError: ignored

In [0]:
# download model image file
from google.colab import files
files.download(WEIGHT_FILE_PATH)